## GLMAT: 2D kernel, Elastic Net estimation

In [ ]:
from __future__ import print_function, division
import os
import sys
import numpy as np
import scipy as sp

import mat_neuron._model as mat
from dstrf import strf, mle

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
# model parameters: (α1, α2, ω, τ1, τ2, tref)
matparams = np.asarray([100, 2, 7, 10, 200, 2], dtype='d')
model_dt = 0.5

# data parameters
duration = 100000
n_samples = int(duration / model_dt)
n_assim = 1
n_test = 0

In [ ]:
# STRF: keep this very simple for proof of principle
stim_dt = 10.0
nfreq = 30
ntau  = 30

from scipy.signal import resample
filts = np.load('../../filters.npz')
print(filts.keys())
k1 = resample(filts['bbs'], nfreq, axis=0)[:,:ntau] * 20

sns.heatmap(k1)

In [ ]:
# generate some random data to fit
np.random.seed(1)
stim_dt = 10.0
upsample = int(stim_dt / model_dt)
stim = np.random.randn(nfreq, int(n_samples / (stim_dt / model_dt)))
stim[:,:100] = 0
plt.imshow(stim, aspect='auto')

In [ ]:
# reduced data:
nfreq = 2
k1 = k1[15:17]
stim = stim[15:17]

In [ ]:
def filter_stimulus(S, kernel):
    """Convolve spectrogram S with spectrotemporal kernel. Kernel should not be flipped."""
    nf, nt = S.shape
    X = np.zeros(nt)
    for i in range(nf):
        X += np.convolve(S[i], kernel[i], mode="full")[:nt]
    return X

def predict_spikes(V, params, dt, upsample):
    a1, a2, omega, t1, t2, tref = params
    return mat.predict_poisson(V - omega, (a1, a2), (t1, t2), tref, 
                               dt, upsample)

In [ ]:
data = []
V = filter_stimulus(stim, k1)
for i in range(n_assim + n_test):
    spikes = predict_spikes(V, matparams, model_dt, upsample)
    H = mat.adaptation(spikes, matparams[3:5], model_dt)
    z = np.nonzero(spikes)[0]
    d = {"H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
        }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
ax1.plot(V)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"], i, i + 0.5)
ax1.set_xlim(0, 8000 // upsample)
ax2.set_xlim(0, 8000)
len(data[0]["spike_t"])

## Estimate parameters using pyglmnet

In [ ]:
# combine the trials
spikes = np.stack([d["spike_v"] for d in data], axis=1)
# spikes in the exponential basis set
X_spikes = np.stack([d["H"] for d in data], axis=2)
# generate design matrix for stimulus
X_stim = strf.lagged_matrix(stim, ntau)
# initial guess of strf
sta = strf.correlate(X_stim, spikes)
plt.plot(np.fliplr(strf.as_matrix(sta, ntau)).T)
plt.plot(k1.T)

In [ ]:
lfuns = mle.make_likelihood(X_stim, X_spikes, spikes, stim_dt, model_dt)
w = np.r_[matparams[2], matparams[:2], np.fliplr(k1).ravel()]
lci = lfuns['lci'](w)
plt.plot(lci[:8000])
lfuns['loglike'](w)

In [ ]:
import scipy.optimize as op
w0 = np.r_[0, 0, 0, sta]
w1 = op.fmin_ncg(lfuns['loglike'], w, lfuns['gradient'], 
                 fhess=lfuns['hessian'], maxiter=100)

In [ ]:
print(w1[:3])
rf_est = strf.as_matrix(w1[3:], ntau)
plt.plot(np.fliplr(rf_est).T)
plt.plot(k1.T)